In [4]:
# Run this once in a cell to convert the notebook
!jupyter nbconvert --to script pkpd_model.ipynb

[NbConvertApp] Converting notebook pkpd_model.ipynb to script
[NbConvertApp] Writing 1572 bytes to pkpd_model.py


In [5]:
import numpy as np
import pkpd_model
from scipy.optimize import least_squares
from pkpd_model import PKPDParams, simulate, add_noise

def make_dosing(qd_hours=24, n_doses=5, start=0.0):
    return np.array([start + i*qd_hours for i in range(n_doses)])

def pack_params(p: PKPDParams):
    return np.array([p.ka, p.ke, p.V, p.Emax, p.EC50])

def unpack_params(x, sigma_C=0.05, sigma_E=0.05):
    return PKPDParams(ka=x[0], ke=x[1], V=x[2], Emax=x[3], EC50=x[4],
                      sigma_C=sigma_C, sigma_E=sigma_E)

def objective(x, t, dose_times, dose_amt, C_obs, E_obs):
    p = unpack_params(x)
    sim = simulate(dose_times, dose_amt, t, p)
    resid_C = (sim["C"] - C_obs)
    resid_E = (sim["E"] - E_obs)
    return np.concatenate([resid_C, resid_E])

def run_demo():
    # True parameters (ground truth for synthetic data)
    p_true = PKPDParams(ka=1.2, ke=0.18, V=30.0, Emax=100.0, EC50=1.5,
                        sigma_C=0.05, sigma_E=2.0)
    t = np.linspace(0, 24*6, 300)          # 6 days, every ~0.5 hr
    dose_times = make_dosing(qd_hours=24, n_doses=6, start=0.0)
    dose_amt = 200.0                        # mg

    sim = simulate(dose_times, dose_amt, t, p_true)
    data = add_noise(sim, p_true)

    # Initial guess (intentionally off)
    x0 = np.array([0.8, 0.25, 20.0, 90.0, 2.0])

    bounds = ([1e-3, 1e-3, 1.0, 1.0, 1e-3],
              [5.0,   2.0,  200.0, 500.0, 50.0])

    res = least_squares(objective, x0, bounds=bounds,
                        args=(t, dose_times, dose_amt, data["C"], data["E"]))
    p_fit = unpack_params(res.x)

    print("True:", p_true)
    print("Fit :", p_fit)

    # Save arrays for plotting (or call a plot util)
    np.savez("pkpd_fit_results.npz",
             t=t, dose_times=dose_times, dose_amt=dose_amt,
             C_obs=data["C"], E_obs=data["E"], x_fit=res.x)

if __name__ == "__main__":
    run_demo()


True: PKPDParams(ka=1.2, ke=0.18, V=30.0, Emax=100.0, EC50=1.5, sigma_C=0.05, sigma_E=2.0)
Fit : PKPDParams(ka=1.5859800087514155, ke=1.5188478599984758, V=3.5871206676185237, Emax=331.71038619871956, EC50=0.0017565915769569564, sigma_C=0.05, sigma_E=0.05)
